In [47]:
import pandas as pd 
import os

In [48]:
def rename(table_name,sign_counter):
    decision_sign = table_name.split('_')[2]
    sign_counter[decision_sign] = sign_counter.get(decision_sign,1) + 1
    return f"{decision_sign}{sign_counter.get(decision_sign,1)-1}" 


def generate_sql_template(csv_path):
    
    df = pd.read_csv(csv_path,delimiter=';') 
    table_names = list(df['TABLE_NAME'].value_counts().index.values)
    pf_key = df['COLUMN_NAME'].value_counts().sort_values().tail(1).index.values[0]
    
    table_col_names = {}
    for table_name in table_names:
        col_names = df[df.TABLE_NAME == table_name]['COLUMN_NAME'].values
        table_col_names[table_name] = col_names 
        
    CreateTableName = csv_path.split('.')[0]    
    select_part = f" CREATE OR REPLACE FORCE VIEW DWH_BV.{CreateTableName} AS select " 
    
    for key,value in table_col_names.items():
        for i in range(len(value)):
            el = value[i]
            if el[-2:] in ['SK','BK']:
                value[i] = f""" {el}  AS {el[:-2]+'ID'}  """
            else:
                pass    
        temp_select = ','.join([ f'{key}.{el.strip()}' for el in value])
        select_part = select_part + temp_select + ',' + os.linesep
        
    sign_counter ={'H':1, 'L':1, 'S':1}    
    primary_table = table_names.pop(0)
    rename_primary_table = rename(primary_table,sign_counter)
    join_part = f"from DWH_DV.{primary_table} AS {rename_primary_table}" +  os.linesep
    for table_name in table_names:
        rename_table_name = rename(table_name,sign_counter)
        join_part = join_part + f" inner join DWH_DV.{table_name}  {rename_table_name} on {rename_primary_table}.{pf_key} = {rename_table_name}.{pf_key} " +  os.linesep

    sql_template = select_part[:-1] + join_part
    return sql_template
    

In [49]:
csv_path = 'MC_ACTIVITY.csv'
sql_template = generate_sql_template(csv_path)
print(sql_template)

 CREATE OR REPLACE FORCE VIEW DWH_BV.MC_ACTIVITY AS select MC_ACTIVITY_S_MCRM.MCACT_SK  AS MCACT_ID,MC_ACTIVITY_S_MCRM.MCACT_NAME,MC_ACTIVITY_S_MCRM.MCACT_START_DT,MC_ACTIVITY_S_MCRM.MCACT_STATUS,MC_ACTIVITY_S_MCRM.MCACT_CLICK_CNT,MC_ACTIVITY_S_MCRM.MCACT_OPEN_CNT,MC_ACTIVITY_S_MCRM.MCACT_CAMPAIGN_ID,MC_ACTIVITY_S_MCRM.MCACT_CAMPAIGN_NAME,MC_ACTIVITY_S_MCRM.MCACT_CAMPAIGN_DESC,MC_ACTIVITY_S_MCRM.MCACT_PRD_DETAILS,MC_ACTIVITY_S_MCRM.MCACT_PROGRAM_NAME,MC_ACTIVITY_S_MCRM.MCACT_PROGRAM_DESC,MC_ACTIVITY_S_MCRM.MCACT_RECORD_TYPE_NAME,MC_ACTIVITY_S_MCRM.MCACT_REGION,
MC_ACTIVITY_L.MCACT_SK  AS MCACT_ID,MC_ACTIVITY_L.MCACT_PRD_SK  AS MCACT_PRD_ID,MC_ACTIVITY_L.MCACT_RETP_SK  AS MCACT_RETP_ID,MC_ACTIVITY_L.MCACT_CUS_SK  AS MCACT_CUS_ID,MC_ACTIVITY_L.MCACT_ACT_SK  AS MCACT_ACT_ID,MC_ACTIVITY_L.MCACT_MCACT_SK  AS MCACT_MCACT_ID,MC_ACTIVITY_L.MCACT_OWNER_EMP_SK  AS MCACT_OWNER_EMP_ID,from DWH_DV.MC_ACTIVITY_S_MCRM AS S1
 inner join DWH_DV.MC_ACTIVITY_L  L1 on S1.MCACT_SK = L1.MCACT_SK 



In [50]:
csv_path = 'VAE_CONSENT.csv'
sql_template = generate_sql_template(csv_path)
print(sql_template)

 CREATE OR REPLACE FORCE VIEW DWH_BV.VAE_CONSENT AS select VAE_CONSENT_S_MAIN.VAECON_SK  AS VAECON_ID,VAE_CONSENT_S_MAIN.VAECON_DT,VAE_CONSENT_S_MAIN.VAECON_CHNL_VALUE,VAE_CONSENT_S_MAIN.VAECON_OPT_EXPIRATION_DT,VAE_CONSENT_S_MAIN.VAECON_OPT_SRC,VAE_CONSENT_S_MAIN.VAECON_OPT_TYPE,VAE_CONSENT_S_MAIN.VAECON_OPT_EVENT_TYPE,
VAE_CONSENT_L.VAECON_SK  AS VAECON_ID,VAE_CONSENT_L.VAECON_CUS_SK  AS VAECON_CUS_ID,VAE_CONSENT_L.VAECON_VAECOT_SK  AS VAECON_VAECOT_ID,VAE_CONSENT_L.VAECON_CAPTURED_EMP_SK  AS VAECON_CAPTURED_EMP_ID,
VAE_CONSENT_S_SIGN.VAECON_SK  AS VAECON_ID,VAE_CONSENT_S_SIGN.VAECON_SIGN_DT,VAE_CONSENT_S_SIGN.VAECON_SIGN_ID,VAE_CONSENT_S_SIGN.VAECON_SIGN_VALUE,
VAE_CONSENT_S_TAIL.VAECON_SK  AS VAECON_ID,VAE_CONSENT_S_TAIL.VAECON_SRC_UPDATE_BY,VAE_CONSENT_S_TAIL.VAECON_SRC_CREATE_BY,VAE_CONSENT_S_TAIL.VAECON_DUO_ID,
VAE_CONSENT_H.VAECON_SK  AS VAECON_ID,VAE_CONSENT_H.VAECON_BK  AS VAECON_ID,from DWH_DV.VAE_CONSENT_S_MAIN AS S1
 inner join DWH_DV.VAE_CONSENT_L  L1 on S1.VAECON_SK = L1

In [51]:
csv_path = 'VAE_TEMPLATE.csv'
sql_template = generate_sql_template(csv_path)
print(sql_template)

 CREATE OR REPLACE FORCE VIEW DWH_BV.VAE_TEMPLATE AS select VAE_TEMPLATE_S_TAIL.VAET_SK  AS VAET_ID,VAE_TEMPLATE_S_TAIL.VAET_VAULT_DOCUMENT_ID,VAE_TEMPLATE_S_TAIL.VAET_VAULT_INSTANCE_ID,VAE_TEMPLATE_S_TAIL.VAET_ANY_PRODUCT_FRAGMENT,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_ALLOWS_DOCUMENTS,VAE_TEMPLATE_S_TAIL.VAET_ALLOWED_DOCUMENT_IDS,VAE_TEMPLATE_S_TAIL.VAET_DOCUMENT_HOST_URL,VAE_TEMPLATE_S_TAIL.VAET_DOCUMENT_ID,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_DOMAIN,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_FROM_ADDRESS,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_FROM_NAME,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_REPLYTO_ADDRESS,VAE_TEMPLATE_S_TAIL.VAET_EMAIL_REPLYTO_NAME,VAE_TEMPLATE_S_TAIL.VAET_LANGUAGE,VAE_TEMPLATE_S_TAIL.VAET_OTHER_DOCUMENT_ID_LIST,VAE_TEMPLATE_S_TAIL.VAET_PI_DOCUMENT_ID,VAE_TEMPLATE_S_TAIL.VAET_SRC_UPDATE_BY,VAE_TEMPLATE_S_TAIL.VAET_SRC_CREATE_BY,VAE_TEMPLATE_S_TAIL.VAET_OWNERID,
VAE_TEMPLATE_L.VAET_SK  AS VAET_ID,VAE_TEMPLATE_L.VAET_PRD_SK  AS VAET_PRD_ID,VAE_TEMPLATE_L.VAET_PAR_PRD_SK  AS VAET_PAR_PRD_ID,VAE_TEMPLATE_L.VA